<a href="https://colab.research.google.com/github/Habibrehman1996/Q2-Projects/blob/main/RAG_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.2 MB/s eta 0:00:00


In [ ]:

from google.colab import userdata


from pinecone import Pinecone, ServerlessSpec



pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "langchain-test-index"  # change if desired
pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
vector = embeddings.embed_query("we are building a RAG Text")

In [ ]:
vector[:5]

[0.04060795158147812,
 -0.04385491460561752,
 -0.05683692544698715,
 -0.021194666624069214,
 0.010449625551700592]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
#Data Save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
len(documents)

10

In [ ]:
from uuid import uuid4
uuid4()

UUID('a0159293-a5e6-4ccc-b891-5c5f01999cb8')

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['e426c702-99bd-4cfa-abad-36ca0de8faa7',
 '67b59885-6f84-46c6-80ea-600e1eb2d77c',
 '291f94e9-5ff5-4af5-8946-805264b9c2f3',
 'ad3981ec-620e-463b-93b3-b7a2b584cedf',
 '64a072bb-fb09-454a-8f7b-d07e76d20842',
 '641a1bfd-11d3-40cb-8dd2-3b5c873f71d8',
 'fe93faae-5b5a-4d8e-bb84-089500d62614',
 'ca7b873a-da32-4bc9-9382-f0afd9b88d3e',
 '52968c1d-4039-4079-97e5-825378d2712a',
 '04610ca2-f1e2-45a8-981b-1f9f99f61c97']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
   k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "what i have in breakfast?"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.723335] I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]
* [SIM=0.551341] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.531009] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.495459] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
def answer_to_user(query: str):

    results = vector_store.similarity_search(query, k=2)

    final_answer = model(results, query)

    return final_answer
